In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import os
from tqdm import tqdm_notebook as tqdm
#from codecs import open
from lxml import etree

from nlppln.utils import out_file_name, create_dirs

In [ ]:
with open('/home/jvdzwaan/data/dbnl_ocr/raw/ocr-with-title-page/_aio001jver01_01.txt') as f:
    ls = f.readlines()

with open('/home/jvdzwaan/data/dbnl_ocr/raw/notes/_aio001jver01_01.txt') as f:
    notes = f.readlines()

print(len(ls))
print(len(notes))

In [ ]:
# remove empty lines from lines
lines = []

for line in ls:
    if line.strip() != '':
        lines.append(line)
print(len(lines))

In [ ]:
notes_text = ''.join(notes)
print(notes_text)

In [ ]:
%%time
from fuzzywuzzy import fuzz

result = pd.DataFrame()
# use difflib
fuzz.SequenceMatcher = difflib.SequenceMatcher
r = []
for l in tqdm(lines):
    r.append(fuzz.partial_ratio(l, notes_text))
result['pratio'] = r

In [ ]:
result.hist(bins=100)

In [ ]:
for idx in list(result[result['pratio'] > 80].index):
    print(idx, lines[idx])

In [ ]:
fuzz.partial_ratio(lines[42], notes[0])

In [ ]:
fuzz.partial_ratio(lines[42], notes_text)

In [ ]:
%%time
# Calculate overlap between remaining notes and lines
from py_stringmatching.similarity_measure.partial_ratio import PartialRatio
pr = PartialRatio()

result = pd.DataFrame()
r = []
for l in tqdm(lines):
    r.append(pr.get_raw_score(l, notes_text))
result['pratio'] = r

In [ ]:
result.hist(bins=100)

In [ ]:
for idx in list(result[result['pratio'] > 80].index):
    print(idx, lines[idx])

In [ ]:
%%time
# match lines to notes
import math
import copy
import edlib

ns = [{'note': n, 'lines': []} for i, n in enumerate(notes)]

In [ ]:
import math

start = 0
stop = math.inf

for n in tqdm(ns):
    for idx in list(result[result['pratio'] > 80].index):
        if idx > start:
            score = pr.get_raw_score(lines[idx], n['note'])
            if score > 80:
                n['lines'].append(idx)
                start = idx
            else:
                break

In [ ]:
for n in ns:
    print(n['note'])
    print('---')
    for l in n['lines']:
        print(l)
    print('-'*80)

In [ ]:
(result[result['pratio'] > 80].index)

In [ ]:
n = 1996
print(lines[n])
print(result.loc[n])

In [ ]:
pr.get_raw_score(lines[n], notes_text)

In [ ]:
pr.get_raw_score(notes_text, lines[n])

In [ ]:
pr.get_raw_score(lines[n], notes[11])

In [ ]:
from fuzzywuzzy import fuzz, StringMatcher
import difflib

#As long as python-Levenshtein is available, that will be used for the following:
print(fuzz.partial_ratio(lines[42], notes[0]))
print(fuzz.partial_ratio(lines[42], notes_text))

#Switch to difflib:
fuzz.SequenceMatcher = difflib.SequenceMatcher
print(fuzz.partial_ratio(lines[42], notes[0]))
print(fuzz.partial_ratio(lines[42], notes_text))